This File is for splitting the Excel sheet (from Steffen) into multiple Excel files. The original Excel sheet contains information about every Experiment done with Mice. The big file is split into smaller files corresponding to the individual Animal IDs.

In [3]:
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Font, Color, Alignment, Border, Side
import os
import numpy as np
from copy import copy

# Set the root directory and the file name of the original Excel file
root_dir = "D:\\Excellsheets"
org_fname = os.path.join(root_dir, "Intrinsic_CA3_database-September_7,_10_08_AM.xlsx")

# Load the workbook and select the active sheet
org_exp_workbook = load_workbook(filename=org_fname)
sheet = org_exp_workbook.active

def row_to_list(sheet, row, bgcolor=False):
    """
    This function takes in an openpyxl sheet object, a row identifier (e.g. "C"), and a boolean flag for whether to return background color or not.
    It returns a list of values from the specified row in the sheet.
    If bgcolor is True, it returns a list of background colors instead of cell values.
    """
    # Define a dictionary to map background color values to color names
    colors = {"FFFF0000": "red", "FFFF9900": "orange","FFFFFF00": "yellow","FF00FF00": "green", "00000000": "white", "FFCCCCCC": "grey"}

    result = []
    # Iterate over cells in the specified row
    for num, cell in enumerate(sheet[row][1:10000]):
        bg_color = cell.fill.bgColor.value
        # If bgcolor is True, get the background color name from the colors dictionary
        value = cell.value if bgcolor==False else colors[bg_color]
        # Append value or background color to result list if cell is not empty
        if cell.value != None:
            result.append(value)
        else:
            break 
    return result

def list_to_count_dict(list):
    """
    This function takes in a list and returns a dictionary with unique elements in the list as keys and their counts as values.
    The dictionary is sorted by count in descending order.
    """
    # Get unique elements and their counts using numpy's unique function
    type, count = np.unique(list, return_counts=True)
    # Get indices that would sort counts in descending order
    count_sort_ind = np.argsort(-count)
    counter_dict = {}
    # Create a dictionary with unique elements as keys and their counts as values
    for num, col in enumerate(type[count_sort_ind]):
        counter_dict[col] = count[count_sort_ind][num]
    return counter_dict

# Get a list of background colors from row A of the sheet
color_list = row_to_list(sheet, "A", bgcolor=True)
# Get a dictionary with unique background colors and their counts
color_count_dict = list_to_count_dict(color_list)

# Print overall color count
print("Overall color count: ")
for col, count in color_count_dict.items():
    print(f"{col}: {count}")

Overall color count: 
green: 206
yellow: 80
white: 68
orange: 25
red: 14


In [5]:
id_list = row_to_list(sheet, "C")
unique_ids = np.unique(id_list)

In [9]:
def find_and_replace_col_index(string_befor, text, replace_string):
    """
    This function takes in three arguments: string_befor, text and replace_string. It finds all occurrences of string_befor in the text and replaces the text between string_befor and the next comma with replace_string. The modified text is then returned.

    :param string_befor: The string to search for in the text
    :type string_befor: str
    :param text: The text to search in
    :type text: str
    :param replace_string: The string to replace the text between string_befor and the next comma with
    :type replace_string: str
    :return: The modified text
    :rtype: str
    """
    # Split the text into parts using string_befor as the separator
    if text != None:
        text_parts = text.split(string_befor)
        
        # Initialize new_age_text with the first part of text_parts
        new_age_text = text_parts[0]
        
        # Iterate over the remaining parts of text_parts
        for part in text_parts[1:]:
            # Find the index of the next comma in part
            index_of_komma = part.find(",")
            
            # Append string_befor, replace_string and the remaining part of part after index_of_komma to new_age_text
            new_age_text += string_befor + replace_string + part[index_of_komma:]
        
        # Return the modified text
        return new_age_text
    else:
        return None

In [10]:
# Splitting Spreadsheet into Animal specific Files
for unique_id in unique_ids:
    # Print the current unique_id being searched for
    print(f"Searching for {unique_id}")

    # Load the original workbook and get the active sheet
    org_exp_workbook = load_workbook(filename=org_fname)
    sheet = org_exp_workbook.active
    
    # Create a new filename for the animal specific file
    own_fname = os.path.join(root_dir, f"Intrinsic_CA3_data_DON-{'0'*(6-len(unique_id[3:]))}{unique_id[3:]}.xlsx")
    if os.path.exists(own_fname):
        print(f"{own_fname} already exists, skipping")
        continue

    # Iterate over the id_list in reverse order
    for line, id in enumerate(id_list[::-1]):
        # If the current id is not equal to unique_id
        if id != unique_id:
            # Calculate the line number and delete the row from the sheet
            line_number = len(id_list) - line
            sheet.delete_rows(idx=line_number+1, amount=1)

    # Get a list of background colors from column A
    color_list = row_to_list(sheet, "A", bgcolor=True)
    
    # Count the occurrences of each color in color_list
    color_count_dict = list_to_count_dict(color_list)
    
    # Write the color_count_dict to cell AW2
    sheet["AW2"] = str(color_count_dict)

    # Correcting Prenatal calculation
    for row_ind, row in enumerate(sheet.iter_rows(min_row=2, max_row=10000)):
        # Break the loop if the first cell in the row is empty
        if row[0].value == None:
            break
        
        # Get the value of cell in column 5 (column E)
        age_text = row[4].value
        
        # Use find_and_replace_col_index function to replace text between (K and next comma with row_ind+2
        new_age_text = find_and_replace_col_index("(K", age_text, str(row_ind+2))
        
        # Use find_and_replace_col_index function to replace text between (B and next comma with row_ind+2
        new_age_text = find_and_replace_col_index("(B", new_age_text, str(row_ind+2))

        # Write new_age_text to cell in column 5 (column E)
        sheet.cell(column = 5, row=row_ind+2).value = new_age_text

    # Save the modified workbook to own_fname
    org_exp_workbook.save(filename=own_fname)

print("Finished")

Searching for DON10473
Searching for DON10477
Searching for DON2865
Searching for DON3165
Searching for DON3343
Searching for DON6084
Searching for DON6085
Searching for DON6087
Searching for DON8497
Searching for DON8498
Searching for DON8499
Searching for DON9191
Searching for DON9192
Finished
